# PPD: OpenMP / *tasks*

Hélio - DC/UFSCar - 2023

# Tasks: criando tarefas sob demanda

O modelo tradicional de programação com *threads* em OpenMP trata da criação de regiões paralelas, executadas por times de *threads*, e da eventual divisão de trabalho (*worksharing*) da região paralela entre as *threads* do time.

Nesses casos, observa-se que:

* O número de *threads* de uma região é pré-definido na criação dessa região paralela, e pode ser consultado pelas *threads* com a chamada *omp_get_num_threads*();
* Cada *thread* pode saber qual é seu identificador lógico, obtido com a chamada *omp_get_thread_num*();
* Programador preocupa-se principalmente com a divisão de carga entre as tarefas:
* Programador pode definir como será a divisão de iterações de um *loop* (*schedule*);
* Programador pode tomar decisões sobre o que executar em cada *thread*, em função do número lógico de cada uma delas no time.


Ao criar uma região paralela, programador define quantas *threads* haverá neste time. As *threads* desta região paralela irão **replicar** a execução do bloco de código, ou **dividir** a execução das iterações de um *loop*, no caso de *parallel for*, ou **dividir** a execução das seções de código, no caso de *parallel sections*.

Além de permitir a criação de tarefas implícitas, associadas às *threads* de um time numa região paralela, OpenMP permite a **criação de tarefas** sob demanda, dinamicamente, de maneira circunstancial ou recursiva, sem saber a priori quantas tarefas serão necessárias. Isso é feito com a diretiva ***task***.

Na programação com tarefas (*tasks*), programador concentra-se em como particionar o código em blocos, que podem ser executados em paralelo; ou seja, em quais trechos de código podem ser transformados em tarefas independentes.

Nesse modelo, cabe ao sistema em tempo de execução determinar como serão o escalonamento e a execução das tarefas (*tasks*) criadas.

Resumidamente, uma *task* é um trecho de código a mais para ser executad0 por alguma das *threads* de um time já existente. Ou seja, a chamada *task* **não cria uma nova *thread*** para o time atual, mas apenas insere uma atividade a mais (tarefa) na lista de tarefas que o time de *threads* da região paralela atual tem para executar.


**task construct**

A diretiva *task* é usada dentro de uma região paralela e define uma tarefa específica. Essa tarefa pode ser executada pela *thread* que encontrar essa diretiva, ou deferida para execução por qualquer outra *thread* no time de *threads* corrente.

Quando uma *task* é criada, se houver alguma *thread* ociosa no time da região paralela atual, a task pode ser executada imediatamente. Caso contrário, fica a critério do sistema em tempo de execução de OpenMP determinar quando esta task será executada.

```
#pragma omp task [clause ...] newline
   structured_block

clauses:
   if (scalar expression)
   untied
   default (shared | none)
   private (list)
   firstprivate (list)
   shared (list)
   final (scalar-expression)
   mergeable
   depend(dependence-type : list)
   priority(priority-value)
```

**Cláusulas da diretiva task**

* **untied**: por padrão, uma *task* é executada pela *thread* que a criou. Quando a cláusula untied é usada, contudo, isso indica que a task pode ser executada por qualquer thread do time. Ao usar cláusula untied, é preciso cuidado com variáveis privadas. Contexto dessas variáveis pode mudar se task for executada por outra tarefa.
* **mergeable**: quando presente, essa cláusula indica que a implementação pode juntar os ambientes de dados das tasks.
* **final** (expr): serve para indicar se a task sendo criada será a última. Resumidamente, quando a criação de tarefas é usada de forma recursiva, essa cláusula permite especificar um nível máximo de profundidade para criação de novas tarefas. 
* **depend** (dependence-type : list): indica relações de dependência entre tasks ou iterações (ver detalhes no manual.)

Exemplo do uso de tasks:

In [ ]:
%%writefile t1.c 

#include <stdio.h>
#include <time.h>
#include <stdlib.h>

#define NUM 10000000
#define MIN_BLK 1024

int _vet[NUM];


int
calc(int start, int finish)
{
   int i, dif;
   int sum = 0, sum1, sum2;

   if (finish-start <= MIN_BLK) {    // calcula
      // printf("Calculando %d -> %d\n", start, finish);

      for (i=start; i < finish; i++)
        sum+=_vet[i];

   } else {                           // divide, criando novas tasks
      
      // printf("Dividindo %d -> %d\n", start, finish);

      dif = finish - start;

      // variável de retorno deve ser compartilhada para que o valor retornado 
      // não seja salvo em uma nova variável da nova task

      #pragma omp task shared(sum1)
      sum1 = calc(start, start + dif / 2);

      #pragma omp task shared(sum2)
      sum2 = calc(finish - dif / 2, finish); 

      // Taskwait funciona como uma barreira para tasks, fazendo com que o fluxo de execução 
      // seja pausado até que as tasks tenham sido completadas. Taskwait faz com 
      // que threads suspendam o que estavam fazendo e passam a atuar nas tasks da fila. 
      // The taskwait construct specifies a wait on the completion of child tasks of the current task.
      #pragma omp taskwait 
      // Once all tasks have be    }   // Fim das seçõesen processed, threads resume their normal execution flow. 

      sum = sum1 + sum2; 
  } 
  return sum; 
}
 
int main() 
{ 
  int i, sum; 
 
  // inicia vetor com valores aleatorios 0..9
  srand(time(NULL));
  for (i=0; i< NUM; i++)
    _vet[i]=rand()%10;

  #pragma omp parallel     // cria um time de threads
  { 
    // Questão: queremos ter um time de threads, que irá executar as tarefas 
    // que criarmos nas chamadas recursivas. Contudo, é preciso fazer apenas 1
    // chamada à função de cálculo. Isso é feito com a diretiva single()

    #pragma omp single 
      sum = calc (0, NUM); 
  } 

  printf("Soma: %d\n", sum); 

  return 0; 
}

Writing t1.c


In [ ]:
! gcc -Wall t1.c -o t1 -fopenmp && time ./t1 

Soma: 44976233

real	0m0.224s
user	0m0.202s
sys	0m0.030s
